In [1]:
import QuantLib as ql

class DFXForward:
    
    def __init__(self, evDate, effective, maturity, position, CCYComm, NPAComm, 
                 CCYTerm, NPATerm, FXSpot, h_discTSComm, h_discTSTerm):
        self._ValueDate = evDate
        self._Effective = effective
        self._Maturity = maturity
        self._Position = position
        self._CCYComm = CCYComm
        self._NPAComm = NPAComm
        self._CCYTerm = CCYTerm
        self._NPATerm = NPATerm
        self._FXSpot = FXSpot
        self._H_discTSComm = h_discTSComm
        self._H_discTSTerm = h_discTSTerm
        
    def NPV(self):
        DfComm = self._H_discTSComm.discount(maturity)
        DfTerm = self._H_discTSTerm.discount(maturity)
        PVComm = DfComm * self._NPAComm
        PVTerm = DfTerm * self._NPATerm
        self._Value = self._Position * (PVComm * self._FXSpot - PVTerm)
        return self._Value

In [2]:
settings = ql.Settings.instance()
evDate = ql.Date(11, 7, 2011)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
DC365 = ql.Actual365Fixed()
DC360 = ql.Actual360()
settlementDays = 0
refDate = Cal.advance(evDate, settlementDays, ql.Days, ql.Following, False)
BDC = ql.Unadjusted

In [3]:
f = 0.01
discTSComm = ql.FlatForward(settlementDays, Cal, f, DC360, ql.Simple, ql.Annual)
h_discTSComm = ql.YieldTermStructureHandle(discTSComm)

In [4]:
r = 0.02
discTSTerm = ql.FlatForward(settlementDays, Cal, r, DC365, ql.Simple, ql.Annual)
h_discTSTerm = ql.YieldTermStructureHandle(discTSTerm)

In [5]:
effective = ql.Date(11, 7, 2011)
maturity = ql.Date(11, 1, 2012)
position = 1.0
NPAComm = 1000
CCYComm = 'USD'
NPATerm = 28000
CCYTerm = 'TWD'
FXSpot = 27.9

In [6]:
FWD = DFXForward(refDate, effective, maturity, position, CCYComm, NPAComm, 
                 CCYTerm, NPATerm, FXSpot, h_discTSComm, h_discTSTerm)
Value = FWD.NPV()
print(Value)

37.60870116406659


In [7]:
FXSpot_1 = 1.01 * FXSpot
FWD_1 = DFXForward(refDate, effective, maturity, position, CCYComm, NPAComm, 
                   CCYTerm, NPATerm, FXSpot_1, h_discTSComm, h_discTSTerm)
Value_1 = FWD_1.NPV()
print(Value_1)

315.1899525458939


In [8]:
Sensitivity = (Value_1 - Value) / 0.01
RW = 0.15
WS = RW * Sensitivity

In [9]:
FXSpot_Up = 1.15 * FXSpot
FWD_Up = DFXForward(refDate, effective, maturity, position, CCYComm, NPAComm, 
                    CCYTerm, NPATerm, FXSpot_Up, h_discTSComm, h_discTSTerm)
Value_Up = FWD_Up.NPV()
print(Value_Up)

4201.327471891458


In [10]:
Curv_Up = -((Value_Up - Value) - (RW * Sensitivity))
Curv_Up

1.8189894035458565e-11